In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split 
from sklearn.neighbors import KNeighborsClassifier

import Helpers as hlp

In [16]:
churn_now = hlp.load_data('train_month_3_with_target.csv')
churn_1mback = hlp.load_data('train_month_2.csv')
churn_2mback = hlp.load_data('train_month_1.csv')

In [17]:
# Merge all dataframes
#churn_all = pd.merge(churn_2mback, churn_1mback, left_index = True, right_index = True, suffixes=('_2', '_1'))
#churn_all = pd.merge(churn_all, churn_now, left_index = True, right_index = True, suffixes=(None, None))

In [18]:
# Do One-hot encoding (can be done before splitting)

churn_now = hlp.Onehotencoding(churn_now)

In [20]:
# Drop integers for now (until we rescale them properly)

integers = churn_now.select_dtypes(include="int64").columns
for i in range(len(integers)):
    churn_now = churn_now.drop(columns = [integers[i]])

In [21]:
# Create x and y variables.

x = churn_now.drop(columns=['target'])
y = churn_now['target']

In [22]:
# Split in train, validation and test set

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, stratify=y, random_state=9)
x_train, x_validation, y_train, y_validation = train_test_split(x_train, y_train, test_size=0.2, stratify=y_train, random_state=9)

In [23]:
# Still missing values in 2 features

x_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 40765 entries, 050687917679384bab2f3a5966569e13 to f142ee164d725c3722da45dbd70daff9
Data columns (total 59 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   homebanking_active                 40765 non-null  bool   
 1   has_homebanking                    40765 non-null  bool   
 2   has_insurance_21                   40765 non-null  bool   
 3   has_insurance_23                   40765 non-null  bool   
 4   has_life_insurance_fixed_cap       40765 non-null  bool   
 5   has_life_insurance_decreasing_cap  40765 non-null  bool   
 6   has_fire_car_other_insurance       40765 non-null  bool   
 7   has_personal_loan                  40765 non-null  bool   
 8   has_mortgage_loan                  40765 non-null  bool   
 9   has_current_account                40765 non-null  bool   
 10  has_pension_saving                 40765 non-null  bool   
 11  h

In [24]:
# Replace missing values in customer_since

x_train['customer_since_all'], mean_csa = hlp.missingtomean(x_train['customer_since_all'])
x_train['customer_since_bank'], mean_csb = hlp.missingtomean(x_train['customer_since_bank'])

In [26]:
# Replace missing values in the validation set 

x_validation['customer_since_all'] = x_validation['customer_since_all'].fillna(mean_csa)
x_validation['customer_since_bank'] = x_validation['customer_since_bank'].fillna(mean_csb)

In [29]:
from sklearn.metrics import confusion_matrix
import warnings
warnings.filterwarnings('ignore')

for i in range(1, 6):
    knn = KNeighborsClassifier(n_neighbors=i)
    knn.fit(x_train, y_train)
    predictions = knn.predict(x_validation)
    tn, fp, fn, tp = confusion_matrix(y_validation, predictions).ravel()
    print((tn, fp, fn, tp))

(9638, 248, 288, 18)
(9869, 17, 306, 0)
(9850, 36, 304, 2)
(9886, 0, 306, 0)
(9881, 5, 305, 1)
